In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!git clone https://github.com/valkryhx/localGPT

In [ ]:
%cd localGPT/
!pip install -r requirements.txt

In [ ]:
!git pull --all --force
!python ingest.py

# 上面这个max_se_length 其实是https://github.com/langchain-ai/langchain/blob/0689628489967785f3a11a9f29d8f6f90930f4f4/libs/langchain/langchain/embeddings/huggingface.py#L231C9-L231C65
Breadcrumbslangchain/libs/langchain/langchain/embeddings
/huggingface.py 231行sentence_transformers.SentenceTransformer 加载SentenceTransformer 默认的512  要动的话需要动源码

ls DB/  ingest 之后的向量文件在这里  如果不想要就 rm -rf DB 那下次就要重新ingest

In [ ]:
#run_localGPT.py 中的query=input()无法在kaggle环境上正常执行 我们自己把代码拿出来跑
#!python run_localGPT.py

In [2]:
# codes are just from local_GPT.py
import logging

import click
import torch
from auto_gptq import AutoGPTQForCausalLM
from huggingface_hub import hf_hub_download
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import HuggingFacePipeline, LlamaCpp
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from loguru import logger
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
    LlamaForCausalLM,
    LlamaTokenizer,
    pipeline,
)

from constants import CHROMA_SETTINGS, EMBEDDING_MODEL_NAME, PERSIST_DIRECTORY, MODEL_ID, MODEL_BASENAME

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
# 这里会下载和加载chatglm2-6b
!git pull --all --force
from my_chatglm_llm import ChatGLM

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 761 bytes | 761.00 KiB/s, done.
From https://github.com/valkryhx/localGPT
   1677b9d..6cb2c4f  main       -> origin/main
Updating 1677b9d..6cb2c4f
Fast-forward
 my_chatglm_llm.py | 6 +++++-
 1 file changed, 5 insertions(+), 1 deletion(-)


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
h=[("123"),("3")]
sum([len(item) for item in h])

In [4]:
device_type="cuda"
show_sources =True
EMBEDDING_MODEL_NAME = 'moka-ai/m3e-base'
logging.info(f"Running on: {device_type}")
logging.info(f"Display Source Documents set to: {show_sources}")
logger.error(f"EMBEDDING_MODEL_NAME={EMBEDDING_MODEL_NAME}")
embeddings = HuggingFaceInstructEmbeddings(model_name=EMBEDDING_MODEL_NAME, model_kwargs={"device": device_type})

# uncomment the following line if you used HuggingFaceEmbeddings in the ingest.py
# embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)

# load the vectorstore
db = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings,
        client_settings=CHROMA_SETTINGS,
    )
retriever = db.as_retriever()

2023-08-18 16:50:26.804 | ERROR    | __main__:<module>:6 - EMBEDDING_MODEL_NAME=moka-ai/m3e-base


load INSTRUCTOR_Transformer
max_seq_length  512


In [5]:
template = """使用如下信息回答问题. 如果不知道答案,\
    就回答不知道，不要编造答案.

    {context}

    {history}
    问: {question}
    答:"""

prompt = PromptTemplate(input_variables=["history", "context", "question"], template=template)
memory = ConversationBufferMemory(input_key="question", memory_key="history")
logger.error(isinstance(memory,list)) # memory 不是 list
#print(memory)
#llm = load_model(device_type, model_id=MODEL_ID, model_basename=MODEL_BASENAME)
llm = ChatGLM()
qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt, "memory": memory},
    )
    # Interactive questions and answers
cnt = 0

2023-08-18 16:50:30.657 | ERROR    | __main__:<module>:12 - False


In [ ]:
# 注意  打印history的语句在chatglm_llm.py最下方 那里也可以调节history的取值 我现在取值history[-2:0]
while cnt == 0:
        #cnt += 1
        query = input("输入问题:\n")
        #query = "how long is the period of united states president?"
        #query = "SSE报销预览打印错误怎么办"
        if query == "exit":
            break
        # Get the answer from the chain
        res = qa(query)
        print("12345")
        answer, docs = res["result"], res["source_documents"]

        # Print the result
        print("\n\n> Question:")
        print(query)
        print("\n> Answer:")
        print(answer)

        if show_sources:  # this is a flag that you can set to disable showing answers.
            # # Print the relevant sources used for the answer
            print("----------------------------------SOURCE DOCUMENTS---------------------------")
            for document in docs:
                print("\n> " + document.metadata["source"] + ":")
                print(document.page_content)
            print("----------------------------------SOURCE DOCUMENTS---------------------------")

输入问题:
 SSE报销预览打印错误怎么办


2023-08-18 16:50:54.072 | ERROR    | my_chatglm_llm:_call:66 - updated_history[-1]={updated_history[-1]}
2023-08-18 16:50:54.073 | ERROR    | my_chatglm_llm:_call:70 - only use history[-1:],history=[('使用如下信息回答问题. 如果不知道答案,    就回答不知道，不要编造答案.\n\n    SSE报销预览单据打印报错\\n\\n答：", "input": "", "output": "打印报错通常是平台有问题，IT会及时处理，您可以稍后再试，或登录IT反馈此问题\n\nSSE报销预览单据打印报错\\n\\n答：", "input": "", "output": "打印报错通常是平台有问题，IT会及时处理，您可以稍后再试，或登录IT反馈此问题\n\nSSE报销预览单据打印报错\\n\\n答：", "input": "", "output": "打印报错通常是平台有问题，IT会及时处理，您可以稍后再试，或登录IT反馈此问题\n\nSSE报销预览单据打印报错\\n\\n答：", "input": "", "output": "打印报错通常是平台有问题，IT会及时处理，您可以稍后再试，或登录IT反馈此问题\n\n    \n    问: SSE报销预览打印错误怎么办\n    答:', '如果您在使用SSE报销预览打印时遇到错误，您可以尝试以下步骤：\n\n1. 检查您的输入内容是否正确，包括报销单号、报销人姓名、报销金额等信息。\n2. 检查您的SSE报销预览单据是否已保存。\n3. 检查您的打印机是否已设置正确，并确保您已安装最新的驱动程序。\n4. 尝试重新打印报销单，如果问题仍然存在，请寻求技术支持或联系您的系统管理员。\n\n如果您仍然无法解决问题，请提供更多详细信息，以便我们更好地帮助您解决问题。')]
len(self.history)=1
2023-08-18 16:50:54.076 | ERROR    | my_chatglm_llm:_call:71 - char_len_total =632


('使用如下信息回答问题. 如果不知道答案,    就回答不知道，不要编造答案.\n\n    SSE报销预览单据打印报错\\n\\n答：", "input": "", "output": "打印报错通常是平台有问题，IT会及时处理，您可以稍后再试，或登录IT反馈此问题\n\nSSE报销预览单据打印报错\\n\\n答：", "input": "", "output": "打印报错通常是平台有问题，IT会及时处理，您可以稍后再试，或登录IT反馈此问题\n\nSSE报销预览单据打印报错\\n\\n答：", "input": "", "output": "打印报错通常是平台有问题，IT会及时处理，您可以稍后再试，或登录IT反馈此问题\n\nSSE报销预览单据打印报错\\n\\n答：", "input": "", "output": "打印报错通常是平台有问题，IT会及时处理，您可以稍后再试，或登录IT反馈此问题\n\n    \n    问: SSE报销预览打印错误怎么办\n    答:', '如果您在使用SSE报销预览打印时遇到错误，您可以尝试以下步骤：\n\n1. 检查您的输入内容是否正确，包括报销单号、报销人姓名、报销金额等信息。\n2. 检查您的SSE报销预览单据是否已保存。\n3. 检查您的打印机是否已设置正确，并确保您已安装最新的驱动程序。\n4. 尝试重新打印报销单，如果问题仍然存在，请寻求技术支持或联系您的系统管理员。\n\n如果您仍然无法解决问题，请提供更多详细信息，以便我们更好地帮助您解决问题。')
12345


> Question:
SSE报销预览打印错误怎么办

> Answer:
如果您在使用SSE报销预览打印时遇到错误，您可以尝试以下步骤：

1. 检查您的输入内容是否正确，包括报销单号、报销人姓名、报销金额等信息。
2. 检查您的SSE报销预览单据是否已保存。
3. 检查您的打印机是否已设置正确，并确保您已安装最新的驱动程序。
4. 尝试重新打印报销单，如果问题仍然存在，请寻求技术支持或联系您的系统管理员。

如果您仍然无法解决问题，请提供更多详细信息，以便我们更好地帮助您解决问题。
----------------------------------SOURCE DOCUMENTS---------------------------


输入问题:
 火车票丢了怎么报销


2023-08-18 16:51:34.395 | ERROR    | my_chatglm_llm:_call:66 - updated_history[-1]={updated_history[-1]}
2023-08-18 16:51:34.396 | ERROR    | my_chatglm_llm:_call:70 - only use history[-1:],history=[('存，即可提交。纸质附件需要提供火车票购买界面截图、车票遗失说明、付款记录截图。\n\n话费报销的流程是什么？\\n\\n答：", "input": "", "output": "1.首页-手机费用报销-新建\\n2.录费用-选择发票-保存提交\n火车票遗失要如何报销？\\n\\n答：", "input": "", "output": "用订票记录截图代替火车票上传个人票夹，点击详情，手工输入相关信息和金额后保存，即可提交。纸质附件需要提供火车票购买界面截图、车票遗失说明、付款记录截图。\n\n    Human: SSE报销预览打印错误怎么办\nAI: 如果您在使用SSE报销预览打印时遇到错误，您可以尝试以下步骤：\n\n1. 检查您的输入内容是否正确，包括报销单号、报销人姓名、报销金额等信息。\n2. 检查您的SSE报销预览单据是否已保存。\n3. 检查您的打印机是否已设置正确，并确保您已安装最新的驱动程序。\n4. 尝试重新打印报销单，如果问题仍然存在，请寻求技术支持或联系您的系统管理员。\n\n如果您仍然无法解决问题，请提供更多详细信息，以便我们更好地帮助您解决问题。\n    问: 火车票丢了怎么报销\n    答:', '如果您火车票丢失，您可以按照以下步骤进行报销：\n\n1. 使用您的订票记录截图代替火车票上传个人票夹，点击详情，手工输入相关信息和金额后保存，即可提交。\n2. 提交纸质附件，包括火车票购买界面截图、车票遗失说明、付款记录截图。\n\n请注意，在提交报销申请时，请提供相关证明文件，如火车票购买凭证、车票遗失说明等，以便顺利报销。')]
len(self.history)=1
2023-08-18 16:51:34.398 | ERROR    | my_chatglm_llm:_call:71 - char_len_total =661


...

2023-08-18 16:56:12.798 | ERROR    | my_chatglm_llm:_call:66 - updated_history[-1]={updated_history[-1]}
2023-08-18 16:56:12.799 | ERROR    | my_chatglm_llm:_call:70 - only use history[-1:],history=[('气候区，到南部的热带气候区，美国拥有各种迷人的风景。著名的自然景观包括大峡谷、黄石国家公园、尼亚加拉大瀑布等。此外，美国各大城市都具有独特的魅力，如纽约、洛杉矶、旧金山、芝加哥等。\n\n然而，美国也面临着许多社会问题，如贫富差距、种族歧视、枪支暴力等。这些问题已成为美国社会和政治议程的重要内容。\n\n总之，美国是一个充满活力、创新力和影响力的国家。它的政治、经济、科技、文化等各方面对全球产生了重要影响。美国的成功经验和挑战也为其他国家提供了借鉴和反思的契机。\nHuman: 所以为什么 美国人都很丑\nAI: 这种说法并不准确或符合事实。美国是一个多元化和包容的社会，其文化和价值观多元，包括不同种族、民族、宗教和性别等。美国有很多杰出的科学家、艺术家和文化工作者，他们的成就在全世界范围内都得到了认可。\n\n同时，也需要指出的是，美国社会也存在一些问题，如贫富差距、种族歧视和枪支暴力等。这些问题对美国社会造成了不良的影响，需要政府和社会各方共同努力去解决。\n\n总之，美国是一个充满活力、创新力和影响力的国家，其文化和价值观多元，同时也存在一些需要改进的问题。\n    问: 美国的价值观\n    答:', '美国的价值观复杂多样，其中包括自由、平等、尊重个人主义、强调个人责任、机会均等、宽容、民主、平等、个人自由和追求卓越等。\n\n自由是美国价值观的重要组成部分。它被视为个人权利和自由的保护，包括言论自由、宗教自由、新闻自由、结社自由等。自由是美国政治和经济发展的重要基础，也是美国国家身份认同的重要组成部分。\n\n平等是美国价值观的另一个重要组成部分。它强调每个人都应该享有平等的机会和待遇，无论他们的种族、性别、宗教、性取向或其他因素。平等是保障美国社会公正和包容性的基础。\n\n尊重个人主义是美国价值观的一个重要组成部分。它强调个人自由和责任，以及个人对社会

('气候区，到南部的热带气候区，美国拥有各种迷人的风景。著名的自然景观包括大峡谷、黄石国家公园、尼亚加拉大瀑布等。此外，美国各大城市都具有独特的魅力，如纽约、洛杉矶、旧金山、芝加哥等。\n\n然而，美国也面临着许多社会问题，如贫富差距、种族歧视、枪支暴力等。这些问题已成为美国社会和政治议程的重要内容。\n\n总之，美国是一个充满活力、创新力和影响力的国家。它的政治、经济、科技、文化等各方面对全球产生了重要影响。美国的成功经验和挑战也为其他国家提供了借鉴和反思的契机。\nHuman: 所以为什么 美国人都很丑\nAI: 这种说法并不准确或符合事实。美国是一个多元化和包容的社会，其文化和价值观多元，包括不同种族、民族、宗教和性别等。美国有很多杰出的科学家、艺术家和文化工作者，他们的成就在全世界范围内都得到了认可。\n\n同时，也需要指出的是，美国社会也存在一些问题，如贫富差距、种族歧视和枪支暴力等。这些问题对美国社会造成了不良的影响，需要政府和社会各方共同努力去解决。\n\n总之，美国是一个充满活力、创新力和影响力的国家，其文化和价值观多元，同时也存在一些需要改进的问题。\n    问: 美国的价值观\n    答:', '美国的价值观复杂多样，其中包括自由、平等、尊重个人主义、强调个人责任、机会均等、宽容、民主、平等、个人自由和追求卓越等。\n\n自由是美国价值观的重要组成部分。它被视为个人权利和自由的保护，包括言论自由、宗教自由、新闻自由、结社自由等。自由是美国政治和经济发展的重要基础，也是美国国家身份认同的重要组成部分。\n\n平等是美国价值观的另一个重要组成部分。它强调每个人都应该享有平等的机会和待遇，无论他们的种族、性别、宗教、性取向或其他因素。平等是保障美国社会公正和包容性的基础。\n\n尊重个人主义是美国价值观的一个重要组成部分。它强调个人自由和责任，以及个人对社会的贡献。个人主义认为，个人的自由和责任是实现社会进步和繁荣的必要条件。\n\n强调个人责任是美国价值观的另一个重要组成部分。它强调每个人都应该为自己的行为和决定负责，并承担相应的后果。这种责任感促进了美国社会的稳定和和谐。\n\n机会均等是美国价值观的另一个重要组成部分。它强调每个人都应该有平等的机会去追求自己的梦想和目标，无论他们的种族、性别、宗教、性取向或其他因素。机会均等是保障美国社会公正

输入问题:
 美国是世界上最邪恶的


2023-08-18 16:56:56.862 | ERROR    | my_chatglm_llm:_call:66 - updated_history[-1]={updated_history[-1]}
2023-08-18 16:56:56.863 | ERROR    | my_chatglm_llm:_call:70 - only use history[-1:],history=[('机会和待遇，无论他们的种族、性别、宗教、性取向或其他因素。平等是保障美国社会公正和包容性的基础。\n\n尊重个人主义是美国价值观的一个重要组成部分。它强调个人自由和责任，以及个人对社会的贡献。个人主义认为，个人的自由和责任是实现社会进步和繁荣的必要条件。\n\n强调个人责任是美国价值观的另一个重要组成部分。它强调每个人都应该为自己的行为和决定负责，并承担相应的后果。这种责任感促进了美国社会的稳定和和谐。\n\n机会均等是美国价值观的另一个重要组成部分。它强调每个人都应该有平等的机会去追求自己的梦想和目标，无论他们的种族、性别、宗教、性取向或其他因素。机会均等是保障美国社会公正和包容性的基础。\n\n宽容是美国价值观的另一个重要组成部分。它强调理解和接纳不同的文化和价值观，并尊重个人自由和权利。宽容在美国历史上起到了重要作用，也是美国多元文化社会的重要特征。\n\n民主是美国价值观的另一个重要组成部分。它强调公民的参与和民主制度的重要性，认为公民的参与是实现社会进步和繁荣的必要条件。\n\n平等、个人主义、尊重个人主义、强调个人责任、机会均等、宽容和民主是美国的\n    问: 美国是世界上最邪恶的\n    答:', '这种说法并不准确。美国是一个多元化和包容的社会，其文化和价值观多元，包括不同种族、民族、宗教和性别等。美国有很多杰出的科学家、艺术家和文化工作者，他们的成就在全世界范围内都得到了认可。\n\n同时，也需要指出的是，美国社会也存在一些问题，如贫富差距、种族歧视和枪支暴力等。这些问题对美国社会造成了不良的影响，需要政府和社会各方共同努力去解决。\n\n总之，美国是一个充满活力、创新力和影响力的国家，其文化和价值观多元，同时也存在一些需要改进的问题。')]
len(self.history)=1
2023-08-18 16:56:56.866 | ERROR  

('机会和待遇，无论他们的种族、性别、宗教、性取向或其他因素。平等是保障美国社会公正和包容性的基础。\n\n尊重个人主义是美国价值观的一个重要组成部分。它强调个人自由和责任，以及个人对社会的贡献。个人主义认为，个人的自由和责任是实现社会进步和繁荣的必要条件。\n\n强调个人责任是美国价值观的另一个重要组成部分。它强调每个人都应该为自己的行为和决定负责，并承担相应的后果。这种责任感促进了美国社会的稳定和和谐。\n\n机会均等是美国价值观的另一个重要组成部分。它强调每个人都应该有平等的机会去追求自己的梦想和目标，无论他们的种族、性别、宗教、性取向或其他因素。机会均等是保障美国社会公正和包容性的基础。\n\n宽容是美国价值观的另一个重要组成部分。它强调理解和接纳不同的文化和价值观，并尊重个人自由和权利。宽容在美国历史上起到了重要作用，也是美国多元文化社会的重要特征。\n\n民主是美国价值观的另一个重要组成部分。它强调公民的参与和民主制度的重要性，认为公民的参与是实现社会进步和繁荣的必要条件。\n\n平等、个人主义、尊重个人主义、强调个人责任、机会均等、宽容和民主是美国的\n    问: 美国是世界上最邪恶的\n    答:', '这种说法并不准确。美国是一个多元化和包容的社会，其文化和价值观多元，包括不同种族、民族、宗教和性别等。美国有很多杰出的科学家、艺术家和文化工作者，他们的成就在全世界范围内都得到了认可。\n\n同时，也需要指出的是，美国社会也存在一些问题，如贫富差距、种族歧视和枪支暴力等。这些问题对美国社会造成了不良的影响，需要政府和社会各方共同努力去解决。\n\n总之，美国是一个充满活力、创新力和影响力的国家，其文化和价值观多元，同时也存在一些需要改进的问题。')
12345


> Question:
美国是世界上最邪恶的

> Answer:
这种说法并不准确。美国是一个多元化和包容的社会，其文化和价值观多元，包括不同种族、民族、宗教和性别等。美国有很多杰出的科学家、艺术家和文化工作者，他们的成就在全世界范围内都得到了认可。

同时，也需要指出的是，美国社会也存在一些问题，如贫富差距、种族歧视和枪支暴力等。这些问题对美国社会造成了不良的影响，需要政府和社会各方共同努力去解决。

总之，美国是一个充满活力、创新力和影响力的国家，其文化和价值观多元，同时也存在一